In [9]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [29]:
train = pd.read_csv("books/train.csv")
valid = pd.read_csv("books/valid.csv")
test = pd.read_csv("books/test.csv")

In [30]:
train = train[train['label']!=3].copy()
valid = valid[valid['label']!=3].copy()
test = test[test['label']!=3].copy()

In [31]:
train

,asin,review,summary,label,title
0,0001061240,Looking for a Louis Untermeyer book from the ...,Two Stars,2.0,The golden treasury of poetry
1,0001712799,Completly boring!!! Yes it's a childerns book ...,Don't waste your money,1.0,Great Day for Up
2,0002005263,I don't like Hillerman novels. It was chosen ...,Two Stars,2.0,The Sinister Pig
3,0002005263,I have read many of the Hillerman books and en...,Tony missed the mark,2.0,The Sinister Pig
4,0002005263,I really love Hillerman's books. He is one of...,Don't Start with This One!,2.0,The Sinister Pig
...,...,...,...,...,...
179995,0006490344,A must read short enough story for everyone wh...,Break away from your destiny. Fly on your own ...,5.0,Jonathan Livingston Seagull : A Story
179996,0006490344,My brother recommended this book to me and it ...,Our lives told by seagulls,5.0,Jonathan Livingston Seagull : A Story
179997,0006483216,If you like historical fiction you will most l...,Good Historical Fiction,4.0,"The Iron Lance (Celtic Crusades, Book 1)"
179998,0006483216,Awesome Book! High adventure and great writing...,"Pretty, pretty good!",5.0,"The Iron Lance (Celtic Crusades, Book 1)"


In [32]:
train.loc[train["label"]<3,'label']=0
train.loc[train["label"]>3,'label']=1
# train.loc[train["label"]==3,'label']=1

valid.loc[valid["label"]<3,'label']=0
valid.loc[valid["label"]>3,'label']=1
# valid.loc[valid["label"]==3,'label']=1

test.loc[test["label"]<3,'label']=0
test.loc[test["label"]>3,'label']=1
# test.loc[test["label"]==3,'label']=1

In [62]:
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = "".join(x.split("<br />"))
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
#     x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [33]:
train['clean_text'] = train.review.astype(str).apply(text_preproc)
valid['clean_text'] = valid.review.astype(str).apply(text_preproc)
test['clean_text'] = test.review.astype(str).apply(text_preproc)

In [57]:
train5 = train[train.label==5].copy()
train1 = train[train.label==1].copy()
train = pd.concat([train5,train1])

In [58]:
valid5 = valid[valid.label==5].copy()
valid1 = valid[valid.label==1].copy()
valid = pd.concat([valid5,valid1])

In [59]:
test5 = test[test.label==5].copy()
test1 = test[test.label==1].copy()
test = pd.concat([test5,test1])

In [60]:
train.loc[train["label"]==1,'label']=0
train.loc[train["label"]==5,'label']=1
# train.loc[train["label"]==3,'label']=1

valid.loc[valid["label"]==1,'label']=0
valid.loc[valid["label"]==5,'label']=1
# valid.loc[valid["label"]==3,'label']=1

test.loc[test["label"]==1,'label']=0
test.loc[test["label"]==5,'label']=1
# test.loc[test["label"]==3,'label']=1

In [61]:
train

,asin,review,summary,label,title
120000,0001713353,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,1.0,"King, the Mice and the Cheese (Beginner Books)"
120001,0001713353,The kids loved it!,Five Stars,1.0,"King, the Mice and the Cheese (Beginner Books)"
120002,0001713353,My students (3 & 4 year olds) loved this book!...,Five Stars,1.0,"King, the Mice and the Cheese (Beginner Books)"
120003,0001713353,LOVE IT,Five Stars,1.0,"King, the Mice and the Cheese (Beginner Books)"
120004,0001713353,Great!,Five Stars,1.0,"King, the Mice and the Cheese (Beginner Books)"
...,...,...,...,...,...
59988,0060755334,"The most admired character in the book, Dean, ...",Harmful to women.....and men,0.0,On The Road CD Unabridged
59990,0060761288,and am grateful for not spending too much. wh...,"Well, I bought it used....",0.0,100 People Who Are Screwing Up America (And Al...
59991,0060761288,"As one who is somewhat progressive (or ""libera...","Biased, but also provides no solutions",0.0,100 People Who Are Screwing Up America (And Al...
59994,0060751967,I guess I have read too many books with depth ...,Disappointed,0.0,How to Marry a Millionaire Vampire (Love at St...


In [34]:
voc_size = 10000
onehot_repr = []
onehot_repr=onehot_repr+[one_hot(sents,voc_size)for sents in train["clean_text"].astype(str)] 
print(len(onehot_repr))
onehot_reprv = []
onehot_reprv=onehot_reprv+[one_hot(sents,voc_size)for sents in valid["clean_text"].astype(str)] 
print(len(onehot_reprv))
onehot_reprt = []
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in test["clean_text"].astype(str)] 
print(len(onehot_reprt))

120000
20000
21964


In [35]:
sent_length=50
train_x=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
train_y = train.label
print(train_x)
print(train_x.shape, train_y.shape)

val_x=pad_sequences(onehot_reprv,padding='pre',maxlen=sent_length)
val_y = valid.label
print(val_x)
print(val_x.shape, val_y.shape)

test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
test_y = test.label
print(test_x)
print(test_x.shape, test_y.shape)

[[   0    0    0 ... 2449  136 9003]
 [   0    0    0 ... 5943 9003 8663]
 [   0    0    0 ... 8351 2449 8685]
 ...
 [   0    0    0 ... 9996 9292 2449]
 [   0    0    0 ... 3549 8923 7653]
 [   0    0    0 ... 7273 6643 2619]]
(120000, 50) (120000,)
[[9274 4309 4369 ... 5274  260 4385]
 [4228 4942 2449 ... 1759 1530 8889]
 [8831 1536 6235 ... 8065 4663 9668]
 ...
 [1882 9920 2148 ... 2449 6825 8162]
 [   0    0    0 ... 2690 9073  906]
 [   0    0    0 ...    0 9995 2449]]
(20000, 50) (20000,)
[[   0    0    0 ... 5086 7096 1812]
 [8136  485 1685 ...  990 6944   43]
 [   0    0    0 ... 9003 6358 2277]
 ...
 [   0    0    0 ... 5418 2817 7096]
 [   0    0    0 ... 9462 2259 5003]
 [   0    0    0 ...    0 3355 2449]]
(21964, 50) (21964,)


In [36]:
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,366,593
Trainable params: 1,366,593
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(train_x, train_y, batch_size=256, 
          epochs=10,validation_data=(val_x,val_y))

Train on 120000 samples, validate on 20000 samples
Epoch 1/10
120000/120000 [==============================] - 79s 656us/sample - loss: 0.3423 - accuracy: 0.8490 - val_loss: 0.3719 - val_accuracy: 0.8432
Epoch 2/10
120000/120000 [==============================] - 74s 615us/sample - loss: 0.2591 - accuracy: 0.8958 - val_loss: 0.3832 - val_accuracy: 0.8449
Epoch 3/10
120000/120000 [==============================] - 74s 615us/sample - loss: 0.2268 - accuracy: 0.9109 - val_loss: 0.3696 - val_accuracy: 0.8479
Epoch 4/10
120000/120000 [==============================] - 75s 622us/sample - loss: 0.1983 - accuracy: 0.9233 - val_loss: 0.4151 - val_accuracy: 0.8385
Epoch 5/10
120000/120000 [==============================] - 74s 615us/sample - loss: 0.1716 - accuracy: 0.9349 - val_loss: 0.4559 - val_accuracy: 0.8335
Epoch 6/10
120000/120000 [==============================] - 74s 614us/sample - loss: 0.1476 - accuracy: 0.9448 - val_loss: 0.5422 - val_accuracy: 0.8277
Epoch 7/10
120000/120000 [=====

In [38]:
model.evaluate(test_x, test_y)

21964/21964 [==============================] - 17s 786us/sample - loss: 0.8671 - accuracy: 0.8155


[0.8671192114457333, 0.81547076]

In [39]:
model.save("books/model.h5")

#

# Word2Vec

#

In [50]:
def text_preproc(x):
    x = x.lower()
#     x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = "".join(x.split("<br />"))
#     x = x.encode('ascii', 'ignore').decode()
#     x = re.sub(r'https*\S+', ' ', x)
#     x = re.sub(r'@\S+', ' ', x)
#     x = re.sub(r'#\S+', ' ', x)
#     x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
#     x = re.sub(r'\w*\d+\w*', '', x)
#     x = re.sub(r'\s{2,}', ' ', x)
    return x

In [6]:
train.review = train.review.astype(str).apply(text_preproc)

In [71]:
x= train.clean_text.copy()
# sen = [nltk.sent_tokenize(s) for s in x]
sen = []
for i in range(len(x)):
    sen += nltk.sent_tokenize(x.iloc[i])
sen

['king, mice cheese nancy gurney excellent children book.',
 'one well remember childhood purchased daughter loves it.',
 'it king trouble rude mice eating cheese.',
 'consults wise men suggest cats chase away mice.',
 'cats become nuisance, wise men recommend king bring dogs chase cats away.',
 'cycle goes mice finally brought back chase away elephants, brought chase away lions that chased away dogs.',
 'the story ends compromise friendship mice king.',
 'story also teaches cause effect relationships.',
 'the pictures accompany story humorous memorable.',
 'thrilled discover back print.',
 '*highly* recommend children ages .',
 'kids loved it!',
 'students ( & year olds) loved book!',
 'definitely recommend teachers.',
 'love',
 'great!',
 'king problems.',
 'fortunately, wise men help solve problems.',
 'unfortunately, wise men turn one problem another problem.',
 'so, top problems wise men problem too.',
 'who wants mice place; naughty mice that?',
 'many food, even beard--you beard

In [75]:
all_words = [nltk.word_tokenize(sent) for sent in sen]
model = Word2Vec(all_words, min_count=100)

In [76]:
words = model.wv.vocab
words

{'king': <gensim.models.keyedvectors.Vocab at 0x231d79ea0f0>,
 ',': <gensim.models.keyedvectors.Vocab at 0x231d79ea1d0>,
 'excellent': <gensim.models.keyedvectors.Vocab at 0x231d79ea208>,
 'children': <gensim.models.keyedvectors.Vocab at 0x231d79ea278>,
 'book': <gensim.models.keyedvectors.Vocab at 0x231d79ea2b0>,
 '.': <gensim.models.keyedvectors.Vocab at 0x231d79ea320>,
 'one': <gensim.models.keyedvectors.Vocab at 0x231d79ea358>,
 'well': <gensim.models.keyedvectors.Vocab at 0x231d79ea390>,
 'remember': <gensim.models.keyedvectors.Vocab at 0x231d79ea3c8>,
 'childhood': <gensim.models.keyedvectors.Vocab at 0x231d79ea400>,
 'purchased': <gensim.models.keyedvectors.Vocab at 0x231d79ea438>,
 'daughter': <gensim.models.keyedvectors.Vocab at 0x231d79ea470>,
 'loves': <gensim.models.keyedvectors.Vocab at 0x231d79ea4a8>,
 'it': <gensim.models.keyedvectors.Vocab at 0x231d79ea4e0>,
 'trouble': <gensim.models.keyedvectors.Vocab at 0x231d79ea550>,
 'eating': <gensim.models.keyedvectors.Vocab at 

In [77]:
sim_words = model.wv.most_similar('plot',topn=50)
sim_words

[('storyline', 0.7990109920501709),
 ('story', 0.7130266427993774),
 ('plots', 0.6827453374862671),
 ('dialog', 0.6143460273742676),
 ('storylines', 0.6100950241088867),
 ('pace', 0.5482267141342163),
 ('action', 0.5449730157852173),
 ('dialogue', 0.5345233678817749),
 ('backstory', 0.533334493637085),
 ('pointless', 0.5237663984298706),
 ('narrative', 0.5208964347839355),
 ('characterizations', 0.5179259181022644),
 ('believable', 0.5060872435569763),
 ('disjointed', 0.48907825350761414),
 ('resolution', 0.48522037267684937),
 ('tension', 0.4850767254829407),
 ('slow', 0.4823295474052429),
 ('contrived', 0.480756551027298),
 ('one-dimensional', 0.480142205953598),
 ('predictable', 0.47950711846351624),
 ('suspense', 0.47876858711242676),
 ('repetition', 0.4779379367828369),
 ('seeming', 0.4762042164802551),
 ('pacing', 0.4693613648414612),
 ('climax', 0.4688490629196167),
 ('exposition', 0.4653405249118805),
 ('plotting', 0.4644126892089844),
 ('surprises', 0.46063363552093506),
 ('tr

In [81]:
wordsList = [word[0] for word in sim_words]
tagged = nltk.pos_tag(wordsList)
mean_list = [tag[0] for tag in tagged if tag[1] == "NN"]
print(mean_list)

['story', 'pace', 'action', 'dialogue', 'backstory', 'pointless', 'resolution', 'tension', 'suspense', 'repetition', 'exposition', 'detail']


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize 
import numpy as np
import re
import string
import sys
from nltk.corpus import stopwords

In [8]:
df1 = pd.read_csv("books/twenty_lakh.csv")
# df1 = df1.loc[:100000].copy()
df1

,Unnamed: 0,Unnamed: 0.1,review,summary,text
0,0,0,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,"king, mice cheese nancy gurney excellent child..."
1,1,1,The kids loved it!,Five Stars,kids loved it!
2,2,2,My students (3 & 4 year olds) loved this book!...,Five Stars,students (3 & 4 year olds) loved book! defini...
3,3,3,LOVE IT,Five Stars,love
4,4,4,Great!,Five Stars,great!
...,...,...,...,...,...
1999996,1999996,1499996,"I'm a great admirer of Willian Boyd, a person ...",a very different opinion,"i'm great admirer willian boyd, person enjoyed..."
1999997,1999997,1499997,A wonderful rampage through the twentieth cent...,Shades of Tristram Shandy (Stern) and Tom Jone...,wonderful rampage twentieth century vein best ...
1999998,1999998,1499998,William Boyd is a terrific storyteller. His p...,another sweeping saga by Boyd fully entertains..,william boyd terrific storyteller. prose high...
1999999,1999999,1499999,This fictional memoir displays Boyd's consumma...,An Outstanding Fictional Memoir,fictional memoir displays boyd's consummate sk...


In [48]:
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
#     x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = " ".join(x.split("<br />"))
    x = " ".join(x.split("\n"))
#     x = x.encode('ascii', 'ignore').decode()
#     x = re.sub(r'https*\S+', ' ', x)
#     x = re.sub(r'@\S+', ' ', x)
#     x = re.sub(r'#\S+', ' ', x)
#     x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    return x

In [5]:
df1['text']='A'

In [6]:
df1['text'] = df1.review.astype(str).apply(text_preproc)

In [7]:
# df1.to_csv("books/twenty_lakh.csv")

In [46]:
df1

,Unnamed: 0,Unnamed: 0.1,review,summary,text
0,0,0,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,"king, mice cheese nancy gurney excellent child..."
1,1,1,The kids loved it!,Five Stars,kids loved it!
2,2,2,My students (3 & 4 year olds) loved this book!...,Five Stars,students (3 & 4 year olds) loved book! defini...
3,3,3,LOVE IT,Five Stars,love
4,4,4,Great!,Five Stars,great!
...,...,...,...,...,...
1999996,1999996,1499996,"I'm a great admirer of Willian Boyd, a person ...",a very different opinion,"i'm great admirer willian boyd, person enjoyed..."
1999997,1999997,1499997,A wonderful rampage through the twentieth cent...,Shades of Tristram Shandy (Stern) and Tom Jone...,wonderful rampage twentieth century vein best ...
1999998,1999998,1499998,William Boyd is a terrific storyteller. His p...,another sweeping saga by Boyd fully entertains..,william boyd terrific storyteller. prose high...
1999999,1999999,1499999,This fictional memoir displays Boyd's consumma...,An Outstanding Fictional Memoir,fictional memoir displays boyd's consummate sk...


In [1]:
from gensim.models import Word2Vec

In [10]:
x= df1.text.astype(str)
# sen = [nltk.sent_tokenize(s) for s in x]
sen = []
for i in range(len(x)):
    sen += nltk.sent_tokenize(x[i])
sen

["king, mice cheese nancy gurney excellent children's book.",
 'one well remember childhood purchased daughter loves it.',
 'it king trouble rude mice eating cheese.',
 'consults wise men suggest cats chase away mice.',
 'cats become nuisance, wise men recommend king bring dogs chase cats away.',
 "cycle goes mice finally brought back chase away elephants, brought chase away lions that'd chased away dogs.",
 'the story ends compromise friendship mice king.',
 'story also teaches cause effect relationships.',
 'the pictures accompany story humorous memorable.',
 'thrilled discover back print.',
 '*highly* recommend children ages 2 7.',
 'kids loved it!',
 'students (3 & 4 year olds) loved book!',
 'definitely recommend teachers.',
 'love',
 'great!',
 'king problems.',
 'fortunately, wise men help solve problems.',
 'unfortunately, wise men turn one problem another problem.',
 'so, top problems wise men problem too.',
 'who wants mice place; naughty mice that?',
 'many food, even beard-

In [19]:
all_words = [nltk.word_tokenize(sent) for sent in sen]

In [24]:
with open('books/all_words.txt', 'w') as f:
    for item in all_words:
        f.write("%s\n" % item)

In [25]:
with open('books/all_words.txt', 'r') as f:
    myNames = [line.strip() for line in f]
len(myNames)

MemoryError: 

In [26]:
model = Word2Vec(all_words, min_count=500)

In [30]:
model.save("books/W2V.model")

In [33]:
modelX = Word2Vec.load("books/W2V.model")

In [36]:
words = modelX.wv.vocab
'plot' in words

True

In [4]:
df = pd.read_csv('books/train.csv')
df

,asin,review,summary,label,title
0,0001061240,Looking for a Louis Untermeyer book from the ...,Two Stars,2.0,The golden treasury of poetry
1,0001712799,Completly boring!!! Yes it's a childerns book ...,Don't waste your money,1.0,Great Day for Up
2,0002005263,I don't like Hillerman novels. It was chosen ...,Two Stars,2.0,The Sinister Pig
3,0002005263,I have read many of the Hillerman books and en...,Tony missed the mark,2.0,The Sinister Pig
4,0002005263,I really love Hillerman's books. He is one of...,Don't Start with This One!,2.0,The Sinister Pig
...,...,...,...,...,...
179995,0006490344,A must read short enough story for everyone wh...,Break away from your destiny. Fly on your own ...,5.0,Jonathan Livingston Seagull : A Story
179996,0006490344,My brother recommended this book to me and it ...,Our lives told by seagulls,5.0,Jonathan Livingston Seagull : A Story
179997,0006483216,If you like historical fiction you will most l...,Good Historical Fiction,4.0,"The Iron Lance (Celtic Crusades, Book 1)"
179998,0006483216,Awesome Book! High adventure and great writing...,"Pretty, pretty good!",5.0,"The Iron Lance (Celtic Crusades, Book 1)"


In [7]:
dfx = df[df['title']=="King, the Mice and the Cheese (Beginner Books)"].copy()
dfx

,asin,review,summary,label,title
60000,0001713353,41 years later:\n\nThe cheese is government ch...,"Outdated, but a good 1960s primer",3.0,"King, the Mice and the Cheese (Beginner Books)"
120000,0001713353,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,5.0,"King, the Mice and the Cheese (Beginner Books)"
120001,0001713353,The kids loved it!,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
120002,0001713353,My students (3 & 4 year olds) loved this book!...,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
120003,0001713353,LOVE IT,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
120004,0001713353,Great!,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
120005,0001713353,Over and over the king has problems. Fortunat...,Not Nice Mice,5.0,"King, the Mice and the Cheese (Beginner Books)"
120006,0001713353,I remember reading my parents' copy of this bo...,One of my favorite kids' stories,5.0,"King, the Mice and the Cheese (Beginner Books)"
120007,0001713353,I am very happy with the book!!! It is one of...,One of our families favorite books!!!,5.0,"King, the Mice and the Cheese (Beginner Books)"
120008,0001713353,"My copy is so tattered, worn and so loved. A ...",probably my favorite children's book of all,5.0,"King, the Mice and the Cheese (Beginner Books)"


In [8]:
aspect = ['plot','character','setting','conflict','style','narration']
model = Word2Vec.load("books/W2V.model")

In [9]:
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = " ".join(x.split("<br />"))
    x = " ".join(x.split("\n"))
#     x = x.encode('ascii', 'ignore').decode()
#     x = re.sub(r'https*\S+', ' ', x)
#     x = re.sub(r'@\S+', ' ', x)
#     x = re.sub(r'#\S+', ' ', x)
#     x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    return x

In [18]:
plot_score = []
char_score = []
setting_score = []
conflict_score = []
style_score = []
narr_score = []

In [19]:
aspect_count = 0
aspect_name = ''
aspect_count1 = 0
aspect_name1 = ''
sentence_raw = nltk.sent_tokenize(df['review'][85400])
sentence = [text_preproc(sen) for sen in sentence_raw]
for sent in sentence:
#     print(sent, end="____________")
    words = nltk.word_tokenize(sent)
    pos = nltk.pos_tag(words)
    for (word,cat) in pos:
#         print(word)
        if (word in model.wv.vocab) == True and cat == 'NN':
#             print('yes')
            for asp in aspect:
                if model.wv.similarity(asp, word) > aspect_count:
                    aspect_count1 = aspect_count
                    aspect_name1 = aspect_name
                    aspect_count = model.wv.similarity(asp, word)
                    aspect_name = asp
#                     print(aspect_count)
#     print((aspect_name,aspect_count),(aspect_name1,aspect_count1))
    onehot_reprx = [one_hot(sent,voc_size)]
    testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
    testing_x.shape
    score = modelX.predict(testing_x)[0][0]
    if(aspect_name=='plot'):
        plot_score.append(2*aspect_count*score)
    elif(aspect_name=='character'):
        char_score.append(2*aspect_count*score)
    elif(aspect_name=='setting'):
        setting_score.append(2*aspect_count*score)
    elif(aspect_name=='conflict'):
        conflict_score.append(2*aspect_count*score)
    elif(aspect_name=='style'):
        style_score.append(2*aspect_count*score)
    elif(aspect_name=='narration'):
        narr_score.append(2*aspect_count*score)
        
    if(aspect_name1=='plot'):
        plot_score.append(2*aspect_count1*score)
    elif(aspect_name1=='character'):
        char_score.append(2*aspect_count1*score)
    elif(aspect_name1=='setting'):
        setting_score.append(2*aspect_count1*score)
    elif(aspect_name1=='conflict'):
        conflict_score.append(2*aspect_count1*score)
    elif(aspect_name1=='style'):
        style_score.append(2*aspect_count1*score)
    elif(aspect_name1=='narration'):
        narr_score.append(2*aspect_count1*score)
    
    aspect_count = 0
    aspect_name = ''
    aspect_count1 = 0
    aspect_name1 = ''

In [24]:
onehot_reprx = []
# s = "i can't believe people are looking for a plot in this film"
s = sentence
onehot_reprx=onehot_reprx+[one_hot(sents,voc_size)for sents in s] 
testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
testing_x.shape
modelX.predict(testing_x)[0][0]

0.98334163

In [22]:
sum(plot_score)/len(plot_score), sum(char_score)/len(char_score), sum(setting_score)/len(setting_score), sum(conflict_score)/len(conflict_score), sum(style_score)/len(style_score), sum(narr_score)/len(narr_score)

(0.534195215680458,
 0.7209141332018741,
 0.09445594434626936,
 0.36069845615286367,
 0.5009064046757752,
 0.25729371192800343)

In [9]:
sentence_raw

['I am a big fan of all mystery and crime novels, so naturally I went out to get this book.',
 'I was sadly disappointed, and almost regret my purchase.',
 "There wasn't the usual umph I feel from his novels.",
 'I do have one spark of good that came out of this.',
 'I took the chance on a new author, I bought her book and loved it.',
 'It was "Waves of Deceit" by Marlene de Velasco.',
 'I would read that one again over this anyday.']

In [4]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# from tensorflow import keras

In [5]:
from tensorflow.keras.models import load_model

In [ ]:
modelX = tensorflow.keras.models.load_model('model')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-ef329dc89d11>", line 1, in <module>
    modelX = tensorflow.keras.models.load_model('model')
NameError: name 'tensorflow' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "D:\Anaconda\envs\tf-g

In [ ]:
voc_size = 10000
sent_length = 50
onehot_reprx = [one_hot("I am disappointed with this book",voc_size)]
testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
testing_x.shape
modelX.predict(testing_x)[0][0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-5ae434fdb034>", line 6, in <module>
    modelX.predict(testing_x)[0][0]
NameError: name 'modelX' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "D:\Anaconda\envs\tf-gpu\lib\site-packages\IPyt

In [26]:
modelX.predict(testing_x)[0][0]

0.9987783